### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import copy
import random
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# 平均値と標準偏差を計算するためのCIFAR100データセットの前処理を定義
pre_transform_cifar100 = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのCIFAR100データセットの読み込み
pre_train_dataset_cifar100 = datasets.CIFAR100(root='/content/data/', download=True, transform=pre_transform_cifar100)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_cifar100 = 0.0
pre_std_cifar100 = 0.0
pre_total_samples_cifar100 = len(pre_train_dataset_cifar100)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_cifar100:
    pre_image, _ = data
    pre_mean_cifar100 += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_cifar100 += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_cifar100 /= pre_total_samples_cifar100
pre_std_cifar100 /= pre_total_samples_cifar100

print("データセット全体の平均値: ", pre_mean_cifar100)
print("データセット全体の標準偏差: ", pre_std_cifar100)

100%|██████████| 169001437/169001437 [00:01<00:00, 102130857.41it/s]


Extracting /content/data/cifar-100-python.tar.gz to /content/data/
データセット全体の平均値:  tensor([0.5071, 0.4866, 0.4409])
データセット全体の標準偏差:  tensor([0.2009, 0.1984, 0.2023])


In [5]:
# 学習用のCIFAR100データセットの前処理を定義
train_transform_cifar100 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar100, std=pre_std_cifar100)
    ])
# テスト用のCIFAR100データセットの前処理を定義
test_transform_cifar100 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar100, std=pre_std_cifar100)
    ])

# 学習用のCIFAR100データセットの読み込み
train_dataset_cifar100 = torchvision.datasets.CIFAR100(root='/content/data/', train=True, transform=train_transform_cifar100, download=True)
# テスト用のCIFAR100データセットの読み込み
test_dataset_cifar100 = torchvision.datasets.CIFAR100(root='/content/data/', train=False, transform=test_transform_cifar100, download=True)

# 学習用のCIFAR100データローダーを作成
train_loader_cifar100 = torch.utils.data.DataLoader(dataset=train_dataset_cifar100, batch_size=512, shuffle=True, num_workers=2)
# テスト用のCIFAR100データローダーを作成
test_loader_cifar100 = torch.utils.data.DataLoader(dataset=test_dataset_cifar100, batch_size=512, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## モデルの実装

In [6]:
# 1×1のサブネットワーク獲得用の畳み込みを定義
def supermaskconv1x1(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3のサブネットワーク獲得用の畳み込みを定義
def supermaskconv3x3(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [7]:
# サブネットワーク獲得用のバッチ正則化として、非アフィン正規化を使用する（学習可能なパラメータを使用しない）
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

In [8]:
# 各重みにスコアを付与したスコアをソートしてtop k%を使用することでサブネットワークを獲得する
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # スコアを複製する
        out = scores.clone()
        # スコアを昇順でソートする
        _, idx = scores.flatten().sort()
        # top k%以下のスコアの数
        j = int((1 - k) * scores.numel())
        # flat_outとoutは同じメモリを参照する（flat_outを変更するとoutにも影響する）
        flat_out = out.flatten()
        # top k%の要素を1にする
        flat_out[idx[j:]] = 1
        # top k%以外の要素を0にする
        flat_out[idx[:j]] = 0
        return out

    @staticmethod
    def backward(ctx, g):
        # 逆伝播時に勾配gをそのまま伝える
        return g, None

In [9]:
# サブネットワーク獲得用の畳み込みを定義
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 重みと同じ形状のスコアを用意
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        # スコアを一様分布で初期化
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))
        # 重みの勾配を無効化する
        self.weight.requires_grad = False

    def _init_conv(self):
        # 重みを一様分布で初期化
        if self.init == 'kaiming_uniform':
            nn.init.kaiming_uniform_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを正規分布で初期化
        elif self.init == 'kaiming_normal':
            nn.init.kaiming_normal_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを符号つき定数で初期化
        elif self.init == 'signed_constant':
            fan = nn.init._calculate_correct_fan(self.weight, mode='fan_out')
            gain = nn.init.calculate_gain('relu')
            std = gain / math.sqrt(fan)
            self.weight.data = self.weight.data.sign() * std

    def set_init(self, init):
        # 初期化手法を設定
        self.init = init

    def set_prune_rate(self, prune_rate):
        # 刈り込み率を設定
        self.prune_rate = prune_rate

    @property
    def clamped_scores(self):
        # スコアとして非負の値を返すようにする（重要度を表す）
        return self.scores.abs()

    def forward(self, x):
        # サブネットワークを獲得
        subnet = GetSubnet.apply(self.clamped_scores, 1 - self.prune_rate)
        # サブネットワークでマスク
        w = self.weight * subnet
        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [10]:
# サブネットワーク獲得用のResidual Blocksを定義
class SupermaskBuildingBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.bn1 = NonAffineBatchNorm(in_channels)
        self.conv1 = supermaskconv3x3(in_channels, out_channels, stride)
        self.bn2 = NonAffineBatchNorm(out_channels)
        self.conv2 = supermaskconv3x3(out_channels, out_channels)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != out_channels or stride > 1:
            self.shortcut = supermaskconv1x1(in_channels, out_channels, stride)
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        # 残差写像と恒等写像の要素毎の和を計算
        out += self.shortcut(x)
        return out

In [11]:
class SupermaskWideResNet(nn.Module):
    def __init__(self, block, depth, k, num_classes=10):
        super().__init__()
        assert (depth - 4) % 6 == 0, "depth should be 6n + 4"
        n = (depth - 4) // 6
        channels = [16, 16 * k, 32 * k, 64 * k]
        self.conv1 = supermaskconv3x3(3, channels[0])
        # Residual Blocks（1)
        self.layer1 = self._make_layer(block, channels[0], channels[1], n)
        # Residual Blocks（2）
        self.layer2 = self._make_layer(block, channels[1], channels[2], n, stride=2)
        # Residual Blocks（3）
        self.layer3 = self._make_layer(block, channels[2], channels[3], n, stride=2)
        self.bn = NonAffineBatchNorm(channels[3])
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(channels[3], num_classes)

        # 重みの初期化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 初期化手法を設定
                m.set_init(init)
                # 刈り込み率を設定
                m.set_prune_rate(prune_rate)
                # 重みを初期化
                m._init_conv()

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, in_channels, out_channels, blocks, stride=1):
        layers = []
        # 最初の Residual Block（stride=stride）
        layers.append(block(in_channels, out_channels, stride))
        # 残りの Residual Block（stride=1）
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [12]:
# モデルを呼び出す関数を定義
def supermaskwideresnet28_10_100():
    return SupermaskWideResNet(SupermaskBuildingBlock, 28, 10, num_classes=100)

## 学習と評価（100 epochs, init=kaiming_uniform）

### prune_rate=0.999

In [14]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.999_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 2.75 %, Loss: 4.3753
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 4.624 %, Loss: 4.2916
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 5.878 %, Loss: 4.0851
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 6.122 %, Loss: 4.0397
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 6.016 %, Loss: 4.0313
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 7.412 %, Loss: 4.0798
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 8.024 %, Loss: 3.9856
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 8.666 %, Loss: 4.0666
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 9.488 %, Loss: 4.0797
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 10.458 %, Loss: 3.8262
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 11.134 %, Los

In [15]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [16]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 6.9 %


In [17]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [18]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.995_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 5.222 %, Loss: 4.1233
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 6.954 %, Loss: 4.0761
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 9.132 %, Loss: 3.8174
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 11.626 %, Loss: 3.7521
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 14.08 %, Loss: 3.4654
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 17.02 %, Loss: 3.5023
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 19.05 %, Loss: 3.3522
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 20.258 %, Loss: 3.1619
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 21.322 %, Loss: 3.5193
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 19.428 %, Loss: 3.1508
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 23.412 %,

In [19]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [20]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 44.35 %


In [21]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [13]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.99_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 6.472 %, Loss: 3.8785
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 11.162 %, Loss: 3.7340
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 13.94 %, Loss: 3.4397
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 15.252 %, Loss: 3.3424
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 18.352 %, Loss: 3.4271
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 20.532 %, Loss: 3.4737
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 23.57 %, Loss: 3.0452
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 28.092 %, Loss: 2.6621
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 31.478 %, Loss: 2.5374
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 34.298 %, Loss: 2.4519
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 37.532

In [14]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 40.29 %


In [16]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [13]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.95_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 7.112 %, Loss: 3.8075
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 11.862 %, Loss: 3.7145
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 14.382 %, Loss: 3.4348
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 18.372 %, Loss: 3.2101
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 22.684 %, Loss: 3.2988
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 27.07 %, Loss: 2.7806
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 31.904 %, Loss: 2.5860
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 35.83 %, Loss: 2.2907
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 40.368 %, Loss: 2.1344
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 43.81 %, Loss: 2.0097
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 46.752 

In [14]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 69.28 %


In [16]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [17]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.9_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.668 %, Loss: 3.8107
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 14.892 %, Loss: 3.4317
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 18.64 %, Loss: 3.2828
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 22.35 %, Loss: 2.8464
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 28.11 %, Loss: 2.5979
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 32.496 %, Loss: 2.4020
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 37.952 %, Loss: 2.2223
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 41.928 %, Loss: 2.0842
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 44.364 %, Loss: 2.2078
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 47.588 %, Loss: 1.9616
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 49.446 

In [18]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [19]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 71.99 %


In [20]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [21]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.7_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 9.15 %, Loss: 3.7624
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 16.094 %, Loss: 3.4719
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 20.67 %, Loss: 3.1743
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 26.012 %, Loss: 2.9561
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 30.534 %, Loss: 2.6975
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 35.592 %, Loss: 2.4979
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 39.516 %, Loss: 2.2634
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 43.506 %, Loss: 2.0523
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 47.17 %, Loss: 1.8556
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 48.508 %, Loss: 1.6510
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 51.056 

In [22]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [23]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 73.22 %


In [24]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [25]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.5_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.98 %, Loss: 3.6950
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 16.07 %, Loss: 3.4277
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 21.148 %, Loss: 3.0678
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 25.836 %, Loss: 2.7758
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 30.736 %, Loss: 2.6158
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 35.144 %, Loss: 2.3934
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 38.814 %, Loss: 2.2260
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 42.634 %, Loss: 2.0877
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 45.158 %, Loss: 1.9709
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 47.208 %, Loss: 1.9977
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 48.544

In [26]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [27]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 72.96 %


In [28]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=kaiming_normal）

### prune_rate=0.999

In [22]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.999_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 2.296 %, Loss: 4.4920
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 4.098 %, Loss: 4.2957
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 5.386 %, Loss: 4.1268
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 6.774 %, Loss: 4.1224
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 7.708 %, Loss: 3.9332
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 8.504 %, Loss: 4.4422
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 9.16 %, Loss: 4.1456
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 10.192 %, Loss: 4.1954
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 10.68 %, Loss: 4.1877
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 11.882 %, Loss: 3.7037
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 12.836 %, Lo

In [23]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [24]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 20.15 %


In [25]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [26]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.995_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 4.936 %, Loss: 4.1052
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 9.568 %, Loss: 3.8426
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 9.328 %, Loss: 3.9405
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 12.406 %, Loss: 3.6272
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 15.69 %, Loss: 3.5524
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 17.73 %, Loss: 3.4060
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 20.386 %, Loss: 3.2410
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 22.174 %, Loss: 3.0749
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 24.232 %, Loss: 2.9934
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 26.234 %, Loss: 2.7781
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 27.272 %

In [27]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [28]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 40.0 %


In [29]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [29]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.99_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 5.362 %, Loss: 4.0066
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 10.338 %, Loss: 3.6502
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 13.626 %, Loss: 3.5728
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 17.234 %, Loss: 3.4425
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 19.68 %, Loss: 3.2386
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 22.8 %, Loss: 3.0825
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 24.952 %, Loss: 2.7614
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 28.444 %, Loss: 2.6762
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 31.484 %, Loss: 2.5549
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 33.958 %, Loss: 2.5317
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 36.8 %,

In [30]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [31]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 52.37 %


In [32]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [33]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.95_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.064 %, Loss: 3.7727
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 14.136 %, Loss: 3.4464
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 16.836 %, Loss: 3.3105
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 21.304 %, Loss: 3.0485
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 26.78 %, Loss: 2.7734
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 31.39 %, Loss: 2.7341
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 35.57 %, Loss: 2.2822
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 38.926 %, Loss: 2.2384
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 41.286 %, Loss: 2.1503
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 44.026 %, Loss: 2.1203
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 45.818 

In [34]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [35]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 64.23 %


In [36]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [37]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.9_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.124 %, Loss: 3.7016
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 14.914 %, Loss: 3.4490
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 19.104 %, Loss: 3.3432
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 23.126 %, Loss: 2.8453
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 28.416 %, Loss: 2.7136
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 32.522 %, Loss: 2.5880
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 37.696 %, Loss: 2.2904
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 41.166 %, Loss: 2.2400
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 43.442 %, Loss: 2.0123
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 46.472 %, Loss: 1.7987
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 48.8

In [38]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [39]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 69.93 %


In [40]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [13]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.7_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.906 %, Loss: 3.6554
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 15.724 %, Loss: 3.5578
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 20.602 %, Loss: 3.0906
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 25.414 %, Loss: 2.7696
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 30.444 %, Loss: 2.9245
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 35.55 %, Loss: 2.3904
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 39.88 %, Loss: 2.1989
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 43.178 %, Loss: 1.9747
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 46.732 %, Loss: 1.8577
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 49.294 %, Loss: 1.7317
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 51.052

In [14]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 72.01 %


In [16]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [45]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.5_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.746 %, Loss: 3.7853
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 15.27 %, Loss: 3.3824
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 19.36 %, Loss: 3.2416
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 24.734 %, Loss: 2.7824
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 30.604 %, Loss: 2.4384
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 34.604 %, Loss: 2.4769
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 38.968 %, Loss: 2.2850
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 42.326 %, Loss: 2.0719
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 44.508 %, Loss: 2.0876
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 46.646 %, Loss: 1.9450
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 48.256

In [46]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [47]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 71.66 %


In [48]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=signed_constant）

### prune_rate=0.999

In [30]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.999_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 2.092 %, Loss: 4.5834
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 3.374 %, Loss: 4.4368
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 3.908 %, Loss: 4.2419
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 4.494 %, Loss: 4.2343
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 4.184 %, Loss: 4.2786
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 6.432 %, Loss: 4.0859
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 8.246 %, Loss: 3.8119
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 9.932 %, Loss: 3.8841
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 11.574 %, Loss: 3.7581
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 13.148 %, Loss: 3.6239
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 14.302 %, L

In [31]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [32]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 26.21 %


In [33]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [34]:

# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.995_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 4.536 %, Loss: 4.0646
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 8.54 %, Loss: 3.7912
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 11.178 %, Loss: 3.7100
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 13.98 %, Loss: 3.5780
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 17.188 %, Loss: 3.4045
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 19.606 %, Loss: 3.3192
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 21.862 %, Loss: 3.1310
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 25.95 %, Loss: 3.0257
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 27.75 %, Loss: 2.9417
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 30.026 %, Loss: 2.9190
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 31.252 %,

In [35]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [36]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 56.6 %


In [37]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [49]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.99_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 4.81 %, Loss: 4.2022
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 9.582 %, Loss: 3.7653
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 12.43 %, Loss: 3.5491
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 16.316 %, Loss: 3.3890
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 21.712 %, Loss: 3.0443
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 26.756 %, Loss: 2.7484
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 29.872 %, Loss: 2.6171
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 32.988 %, Loss: 2.5618
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 36.102 %, Loss: 2.3768
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 38.998 %, Loss: 2.2843
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 41.91 %

In [50]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [51]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 60.97 %


In [52]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [53]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.95_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.316 %, Loss: 3.8394
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 13.57 %, Loss: 3.4689
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 17.41 %, Loss: 3.3065
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 22.246 %, Loss: 3.0176
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 27.054 %, Loss: 2.7994
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 31.722 %, Loss: 2.5952
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 36.83 %, Loss: 2.4459
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 40.466 %, Loss: 2.1808
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 44.042 %, Loss: 1.9787
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 45.888 %, Loss: 1.9043
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 47.618 

In [54]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [55]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 70.25 %


In [56]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [57]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.9_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 8.836 %, Loss: 3.7559
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 14.912 %, Loss: 3.4088
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 19.504 %, Loss: 3.1975
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 24.668 %, Loss: 2.8117
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 30.384 %, Loss: 2.4080
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 35.404 %, Loss: 2.3647
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 40.046 %, Loss: 2.1809
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 43.958 %, Loss: 2.0694
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 46.378 %, Loss: 2.0597
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 48.66 %, Loss: 1.8029
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 51.12

In [58]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [59]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 72.8 %


In [60]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [61]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.7_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 9.25 %, Loss: 3.6797
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 16.466 %, Loss: 3.3602
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 22.086 %, Loss: 3.0407
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 26.684 %, Loss: 2.7252
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 32.168 %, Loss: 2.5273
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 37.112 %, Loss: 2.2350
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 41.282 %, Loss: 2.2669
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 44.354 %, Loss: 1.9932
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 47.406 %, Loss: 1.9930
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 49.416 %, Loss: 1.9356
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 51.45

In [62]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [63]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 74.08 %


In [64]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [13]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR100_100epochs_pr0.5_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 9.188 %, Loss: 3.7388
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 16.102 %, Loss: 3.4024
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 20.962 %, Loss: 3.0404
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 25.994 %, Loss: 2.7349
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 31.138 %, Loss: 2.4850
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 35.994 %, Loss: 2.4963
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 39.87 %, Loss: 2.1323
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 43.584 %, Loss: 2.1142
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 45.856 %, Loss: 2.2664
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 47.938 %, Loss: 1.8622
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 49.43

In [14]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 74.14 %


In [16]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()